In [1]:
# from transformers import BertTokenizer, BertModel
import transformers
import torch
import torch.nn as nn
import numpy
from transformers import BertTokenizer
from transformers import BertModel
from torch.utils.data import TensorDataset, DataLoader

In [74]:
from datasets import load_dataset
dataset = load_dataset("SetFit/sst5")

Repo card metadata block was not found. Setting CardData to empty.


In [15]:
dataset["train"]["label_text"]

Column(['very positive', 'negative', 'negative', 'neutral', 'positive'])

In [92]:
seed = 50
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # if using GPU

    # Ensure deterministic behavior (slower but reproducible)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
set_seed(seed)

In [ ]:
train_loader = DataLoader(dataset["train"], batch_size=16, shuffle=True)
val_loader   = DataLoader(dataset["validation"], batch_size=16)
# print(len(X))
# print(len(y))

In [97]:
class CustomBertClassifier(nn.Module):
    def __init__(self, num_labels=3):  # e.g., 3-class task
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        # self.dropout = nn.Dropout(0.3)
        # self.h1 = nn.Linear(self.bert.config.hidden_size, num_labels * 12)
        # self.h1a = 
        # self.classifier = nn.Linear(num_labels * 12, num_labels)
        # print(num_labels)
        self.classifier = nn.Sequential(
                nn.Dropout(0.3),    # optional
                # nn.Linear(self.bert.config.hidden_size, num_labels * 12),
                # nn.GELU(),          # <-- activation
                # nn.Dropout(0.3),    # optional
                # nn.Linear(num_labels * 12, num_labels)
                nn.Linear(self.bert.config.hidden_size, num_labels)
        )

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        cls_output = outputs.last_hidden_state[:,0,:]  # CLS token
        # x = self.dropout(cls_output)
        logits = self.classifier(cls_output)
        return logits

In [98]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

text = ["I love AI!", "This is terrible..."]
# labels = torch.tensor([1, 0])  # example labels
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# encoding = tokenizer(text, return_tensors='pt', padding=True, truncation=True)

model = CustomBertClassifier(num_labels=5)
model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5, weight_decay=0.01)
criterion = nn.CrossEntropyLoss()

# model.train()

In [ ]:
tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=120)["input_ids"]

In [10]:
device

device(type='cuda', index=0)

In [ ]:
# X = []
# y = []
# for n in dataset["train"]:
#     x = tokenizer(n["sentence"], return_tensors='pt', padding=True, truncation=True)
#     labels = [0, 0, 0]
#     labels[n["label"]] = 1
#     x["label"] = labels
#     X.append(x)
#     # y.append(labels)

In [84]:
def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=128)
def to_device(batch):
    return {k: (v.to(device) if isinstance(v, torch.Tensor) else v) for k, v in batch.items()}
    
datasetMap = dataset
# datasetMap = datasetMap.map(to_device, batched=True, batch_size=len(dataset["train"]))
datasetMap = datasetMap.map(tokenize, batched=True)
datasetMap = datasetMap.rename_column("label", "labels")  # rename for HF models
datasetMap.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

In [100]:
batch_size=32
train_loader = DataLoader(datasetMap["train"], batch_size=batch_size, shuffle=True, num_workers=4)
val_loader   = DataLoader(datasetMap["validation"], batch_size=batch_size, num_workers=4)

In [ ]:
# X = torch.tensor(X)
# y = torch.tensor(y)
# batch_size = 32

# train_loader = DataLoader(
#     # TensorDataset(X, y),
#     X,
#     batch_size=batch_size,
#     shuffle=True,
#     num_workers=0
# )

In [102]:
# GPU training
step = 10
epochs = 15
for epoch in range(epochs):  # 3 epochs seems to be the optimal number
    model.train()
    total_losst = 0
    curBatch = 0
    correctLabelsT = 0
    progressT = 0
    for batch in train_loader:
        # print(batch)
        # break
        optimizer.zero_grad()
        curBatch += 1
        # print(batch)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        # Count how many labels are correctly predicted in this batch
        logitLabels = torch.argmax(logits, dim=1)
        correctLabelsT += (logitLabels == labels).sum()
        progressT += len(logitLabels)
        
        # loss = outputs.loss
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()

        total_losst += loss.item()

        if curBatch % step == 0:
            print(f"Step {progressT}, Loss: {total_losst / curBatch}, correctP: {correctLabelsT / progressT}")
            
    trainLastBatch = curBatch
    

    model.eval()
    with torch.no_grad():
        total_loss = 0
        curBatch = 0
        correctLabels = 0
        progress = 0
        for batch in val_loader:
            # print(batch)
            # break
            optimizer.zero_grad()
            curBatch += 1
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)
            # print(labels)
    
            logits = model(input_ids=input_ids, attention_mask=attention_mask)
            # Count how many labels are correctly predicted in this batch
            logitLabels = torch.argmax(logits, dim=1)
            correctLabels += (logitLabels == labels).sum()
            progress += len(logitLabels)
            # loss = outputs.loss
            loss = criterion(logits, labels)
            # loss.backward()
            # optimizer.step()
    
            total_loss += loss.item()
            # break
            if curBatch % step == 0:
                print(f"Step {progress}, Loss: {total_loss / curBatch}, correctP: {correctLabels / progress}")

        avg_loss = total_losst / len(train_loader)
        print(f"Train Epoch {epoch+1}, Loss: {avg_loss:.4f}, correctP: {correctLabelsT / progressT}")
        avg_loss = total_loss / len(val_loader)
        print(f"Val Epoch {epoch+1}, Loss: {avg_loss:.4f}, correctP: {correctLabels / progress}")

Step 320, Loss: 0.049106602743268016, correctP: 0.984375
Step 640, Loss: 0.050870767375454304, correctP: 0.9828125238418579
Step 960, Loss: 0.04840591934820016, correctP: 0.98333340883255
Step 1280, Loss: 0.05806019314331934, correctP: 0.98046875
Step 1600, Loss: 0.05887166610918939, correctP: 0.9799999594688416
Step 1920, Loss: 0.07467282174620778, correctP: 0.9750000238418579
Step 2240, Loss: 0.07935777713012482, correctP: 0.9723214507102966
Step 2560, Loss: 0.08092421702458523, correctP: 0.9710937738418579
Step 2880, Loss: 0.0808781619131979, correctP: 0.9722222685813904
Step 3200, Loss: 0.07890538932289928, correctP: 0.9724999666213989
Step 3520, Loss: 0.08102125272747468, correctP: 0.9715908765792847
Step 3840, Loss: 0.07991344585704306, correctP: 0.9721354842185974
Step 4160, Loss: 0.07789861602135575, correctP: 0.9728365540504456
Step 4480, Loss: 0.07721087905977453, correctP: 0.9732142686843872
Step 4800, Loss: 0.07636221000303825, correctP: 0.9733333587646484
Step 5120, Loss: 

In [59]:
# Validate
step = 10
model.eval()
with torch.no_grad():
    total_loss = 0
    curBatch = 0
    for batch in val_loader:
        # print(batch)
        # break
        optimizer.zero_grad()
        curBatch += 1
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)
        # print(labels)

        logits = model(input_ids=input_ids, attention_mask=attention_mask)
        print(labels)
        print(logits)
        break
        # loss = outputs.loss
        loss = criterion(logits, labels)
        # loss.backward()
        # optimizer.step()

        total_loss += loss.item()
        # break
        if curBatch % step == 0:
            print(f"Step {curBatch * batch_size}, Loss: {total_loss / curBatch}")

    avg_loss = total_loss / len(val_loader)
    print(f"Epoch {1}, Loss: {avg_loss:.4f}")

tensor([2, 1, 1, 0, 1, 3, 2, 2, 3, 0, 3, 2, 1, 3, 2, 3, 4, 2, 2, 1, 2, 1, 3, 2,
        0, 3, 4, 3, 3, 3, 1, 0], device='cuda:0')
tensor([[ 0.1370,  0.0369, -0.3140, -0.0769,  0.2008],
        [ 0.0680,  0.0955, -0.2299, -0.0780,  0.1754],
        [-0.0983,  0.1429, -0.2284, -0.0066,  0.0506],
        [ 0.0432,  0.1726, -0.2526, -0.0213,  0.1162],
        [ 0.0540,  0.1167, -0.3187, -0.0821,  0.2123],
        [ 0.1089,  0.1473, -0.2561, -0.1369,  0.1871],
        [-0.0385,  0.1535, -0.2466, -0.0070,  0.0550],
        [-0.0123,  0.1271, -0.2191, -0.0330,  0.1465],
        [-0.0088,  0.1197, -0.2667,  0.0277,  0.0696],
        [-0.0237,  0.1514, -0.2483, -0.0647,  0.1297],
        [ 0.0525,  0.1411, -0.2403, -0.0590,  0.1217],
        [-0.0090,  0.1142, -0.2299, -0.0382,  0.1277],
        [-0.1220,  0.1796, -0.1817, -0.0036,  0.0230],
        [ 0.1192,  0.0526, -0.3610,  0.0081,  0.1529],
        [ 0.0293,  0.1438, -0.2093,  0.0523,  0.0152],
        [ 0.0219,  0.1446, -0.1859, -0.0103, 

In [60]:
logitLabels = torch.argmax(logits, dim=1)
matches = (logitLabels == labels).sum()
print(f"{matches} out of {len(logitLabels)}")

6 out of 32


In [ ]:
epochs = 5
step_loss = 0
for n in dataset["train"]:
    optimizer.zero_grad()
    encoding = tokenizer(n["sentence"], return_tensors='pt', padding=True, truncation=True)
    logits = model(encoding['input_ids'], encoding['attention_mask'])
    print(logits)
    labels = [0, 0, 0]
    labels[n["label"]] = 1
    loss = criterion(logits[0], torch.tensor(labels, dtype=torch.float32))
    loss.backward()
    optimizer.step()
    
    print("Loss:", loss.item())

In [ ]:
x = 0
for n in dataset["train"]:
    print(n)
    encoding = tokenizer(n["sentence"], return_tensors='pt', padding=True, truncation=True)
    logits = model(encoding['input_ids'], encoding['attention_mask'])
    print(logits)
    x += 1
    if x > 10:
        break